In [2]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as me
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i{\bf{\mathcal{M}}}(t) \textbf{c}(t)
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

where

1: $A=\alpha B = N(t) \exp(-m t^{1+a}) = I_M^{(0)}$ (a primer orden alpha = 1) \
2: hacer la matriz hermìtica \
3: $C = M N(t) exp(-m t^{1+a})$
    
$K(t,t') = e^{(-i M(t-t'))}$
    
con $N(t) = 1/Tr(\rho(t))$ la calculo con la definición del $\rho_M (t)=  c_M(t) \rho^0_M(t)$
con c_0(t)= 1. 

H_ {0, 2}daggger = H_{0, -2}. 
rho_{M}dagger = rho{-M}

In [3]:
p = .0009; a = .05; M = 500; coherences_t0_pert0 = 1.
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a}
coeff_list_t0 = np.array([coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])])
coeff_list_t0 = coeff_list_t0/(sum(coeff_list_t0))
M_tensor_f= np.array(me.gen_func_even_cohr_M_matrix(param_list, coeff_list_t0, 1, False, False, False))

In [73]:
from scipy.integrate import odeint

def differential_system(cohr_vector_t0, t, parameters):
    """
    This module sets up the system of time-dependent differential
    equations, given by 
    
    dC/dt = M(t)*c(t),
    
    wherein M(t) is the coherence-weight matrix. 
    In accordance with Scipy.Odeint's documentation, this module takes
    as input the following parameters, 
    
    *** 1. cohr_vector_t0: an initial configuration of weights for the coherences at time zero, 
           such that this configuration has trace one.
    *** 2. t: time
    *** 3. parameters: a dictionary containing three parameters:
                        ☼*☼: the total number of coherences in the system,
                        ☼*☼: the chosen p-factor, the weight for the 
                             perturbation Hamiltonian, Sigma. 
                        ☼*☼: the power law factor used in the approximation.
    
    ===> Returns: an array of numbers
    
    """
    return gen_func_even_cohr_M_matrix(parameters = parameters,
                                       init_configurations = cohr_vector_t0, 
                                       timet = t,
                                       closed_boundary_conditions = False,
                                       visualization = False,
                                       as_qutip_qobj = False) @ (cohr_vector_t0)

differential_system(cohr_vector_t0 = coeff_list_t0, 
                    t=1, parameters = param_list, 
                    )

array([0.776, 0.21, 0.448, ..., 0.37, 0.242, 0.185])

In [ ]:
from scipy.integrate import odeint

ts = np.linspace(0.1, 1., 100)
result = odeint(func = differential_system, 
                y0 = coeff_list_t0, 
                t = ts,
                args = ((param_list,)))

plt.plot(result[0])

In [29]:
qutip.Qobj(M_tensor_f)
#assert me.M_matrix_test(M_tensor_f), "ERROR: M-matrix not next-diagonal triangular."

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[0.11900000000000001 0.88 0.0 ... 0.0 0.0 0.0]
 [0.145 -0.217 1.072 ... 0.0 0.0 0.0]
 [0.0 0.135 -0.135 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.151 0.747 0.0]
 [0.0 0.0 0.0 ... 0.096 0.192 0.711]
 [0.0 0.0 0.0 ... 0.0 0.153 0.846]]

In [ ]:
Preguntas:
    
    1. La matrix M tiene que depender con el tiempo me parece, si A, B y C lo hacen
    2. Como tendría que implementar la norma???
    3. Empezar a jugar con los parámetros. 
    
m+2 vs m+2
** reescribir la matriz en tèrminos de los c_pares.

In [45]:
def generating_function_even_cohr_M_matrix(parameters, init_configuration, timespan, 
                                           closed_boundary_conditions = False):
    
    M = parameters["total_no_cohrs"]; p = parameters["p_factor"]; a = parameters["power_law_factor"]
    cm_list = init_configuration
    m_matrix_list = []; t = timespan;
    
    for m in range(M):
        if m == 0:
            m_matrix_list.append(np.array([diag_mm_matrix_elmt(m,t,a)] 
                                    + [A_mmplustwo_matrix_elmt(m+2,t,a)] + [0 for k in range(int((M-3)/2))])) 
        if m == 1:
            local_list = [0] + [diag_mm_matrix_elmt(m,t,a)] + [A_mmplustwo_matrix_elmt(m+2, t, a)]
            local_length = len(np.array(local_list))
            m_matrix_list.append(np.array(local_list + [0 for k in range(int(M/2 + 1)-local_length)]))
        if (m > 1) and (m < M - 2):  
            local_list = [0 for j in range(m-2)]; local_length = len(local_list)
            local_list += ([B_mmminustwo_matrix_elmt(-m-2,t,a)] + [diag_mm_matrix_elmt(m,t,a)] 
                        + [A_mmplustwo_matrix_elmt(m+2,t,a)])
            local_list += [0 for j in range(int(M/2+1)-local_length - 5)]
            m_matrix_list.append(np.array(local_list))
        if m == M - 2: 
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)]  
                                 + [diag_mm_matrix_elmt(m,t,a)] + [0]))
        if m == M - 1:
            m_matrix_list.append(np.array([0 for i in range(int(M/2)-2)] + [B_mmminustwo_matrix_elmt(-m-2,t,a)] + [0] 
                                 + [diag_mm_matrix_elmt(m,t,a)]))
    return (m_matrix_list)


In [72]:
v = [np.random.rand() for i in range(5)]
def example_matrix(t, p):
    return (t+p) * np.identity(5) 

def example_diff_eq(v, t, p):
    return example_matrix(t, p).dot(v)

ts = np.linspace(0., 10., 100)
result = odeint(func = example_diff_eq, 
                y0 = v, 
                t = ts,
                args = (p,))

In [4]:
A = np.array([-1,2,3])
sum(A)

4

In [ ]:
# Si la matrix M(t) = M para todo t, uso la propiedad de semigrupo y ya fue

def semigroup_phit_and_rhot_sol(coeff_list_t0, Mtensor, ts):
    coeff_at_timet_list = []
    coeff_at_timet_list.append(np.array(coeff_list_t0))
    
    for i in range(1, len(ts)-1):
        try: 
            a = (ts[i+1] * Mtensor).expm() * coeff_list_t0[0]
            coeff_at_timet_list.append(a)
        #if (Exception):
        #   continue 
    return coeff_at_timet_list    

# Si la matrix M(t) fuese no constante, odeints

from scipy.integrate import ode
import random

def F(y, t, Mtensor):
    return Mtensor.dot(y)

real_Mtensor = (np.real(Mtensor))
ts1 = np.linspace(0, 500, 200)
result = odeint(func=F, y0=coeff_list_t0, t= ts1 , args=(real_Mtensor,))
plt.figure(figsize=(7, 4))

for i in range(len(total_no_spins)):
    plt.plot(ts1, result[:,i], label=r'$c_{(%d)}(t)$'%i)

plt.xlabel("t")
plt.ylabel("Phi(t)")
plt.legend()

